In [13]:
import datetime
import definitions as d
import geopy.distance
import neural_network as nn
import numpy as np
import os
import pandas as pd

from sklearn.preprocessing import OneHotEncoder


def load_raw_data(path, type):
    files = os.listdir(f"{path}/{type}")
    data = {}
    surfix = f"_{type}.csv"
    for file in files:
        name = file[:file.find(surfix)]
        df = pd.read_csv(f"{path}/{type}/{file}", sep=";")
        data[name] = df
    return data

def get_nearest_cities(city_attributes):
    nearest_cities = {}
    for _, row in city_attributes.iterrows():
        source = (row["Latitude"], row["Longitude"])
        city_dist = []
        for _, row2 in city_attributes.iterrows():
            if row["City"] is row2["City"]:
                continue
            destination = (row2["Latitude"], row2["Longitude"])
            city_dist.append((row2["City"], geopy.distance.geodesic(source, destination).km))
        city_dist.sort(key=lambda x: x[1])
        nearest_cities[row["City"]] = [cd[0] for cd in city_dist[:3]]
    return nearest_cities
    
def load_train():
    dict = load_raw_data("data", "train")
    dict.pop("weather_description")
    for key, df in dict.items():
        dict[key] = df.iloc[12:, :]
    return dict

def load_test():
    dict = load_raw_data("data", "test")
    dict.pop("weather_description")
    for key, df in dict.items():
        dict[key] = df.iloc[:-1, :]
    return dict

def get_normalization_params(raw):
    params = {}
    for key, df in raw.items():
        all = np.reshape(df.to_numpy()[:, 1:], -1)
        params[key] = (np.nanmean(all), np.nanstd(all))
    return params

def to_city_time_vect(raw):
    cities = next(iter(raw.values())).columns[1:]
    hours = next(iter(raw.values()))[["datetime"]]
    ctvs = {c: hours.copy() for c in cities}
    for city in cities:
        for key, df in raw.items():
            ctvs[city][key] = df[[city]]
    return ctvs

def normalize(ctv, params):
    for df in ctv.values():
        for param, ms in params.items():
            mean, std = ms
            df[param] = (df[param] - mean) / std 

def normalize_city_attributes(city_attributes):
    latitude_mean = city_attributes["Latitude"].mean()
    latitude_std = city_attributes["Latitude"].std()

    longitude_mean = city_attributes["Longitude"].mean()
    longitude_std = city_attributes["Longitude"].std()

    city_attributes["Latitude"] = (city_attributes["Latitude"] - latitude_mean) / latitude_std
    city_attributes["Longitude"] = (city_attributes["Longitude"] - longitude_mean) / longitude_std

def to_city_day_vect(ctv, wind_treshold):
    cdv = {}
    for city, df in ctv.items():
        u = 0
        cdr = []
        while u < len(df):
            w = u + 24
            date = df.iloc[u, 0]
            vec = np.reshape(df.iloc[u : w, 1:].to_numpy(), -1)
            temp_mean = df.iloc[u : w]["temperature"].mean()
            wind_cat = int(np.any(df.iloc[u:w]["wind_speed"].to_numpy() > wind_treshold))
            cdr.append((date, vec, temp_mean, wind_cat))
            u = w
        cdv[city] = cdr
    return cdv

def get_city_encoder(cities_attr):
    cities = np.reshape(city_attributes_raw["City"].to_numpy(), (-1, 1))
    cohe = OneHotEncoder()
    cohe.fit(cities)
    return cohe

def get_wind_treshold(souce, params):
    mean, std = params["wind_speed"]
    return (souce - mean) / std

def drop_nan_records(data_set):
    mask = [np.any(np.isnan(val), axis=0) for val in data_set.values()]
    mask = np.vstack(mask)
    mask = np.any(mask, axis=0)
    return {key: val[:, ~mask] for key, val in data_set.items()}

def get_set1(cdv, city_encoder, city_attributes_raw):
    d1 = []
    d2 = []
    d3 = []
    output_temp = []
    output_wind = []
    date = []
    city_one_hot = []
    cord = []

    for city, dv in cdv.items():
        d1 += [r[1] for r in dv[:-4]]
        d2 += [r[1] for r in dv[1:-3]]
        d3 += [r[1] for r in dv[2:-2]]
        output_temp += [r[2] for r in dv[4:]]
        output_wind += [np.hstack((r[3], 1 - r[3])) for r in dv[4:]]
        date_str = [r[0] for r in dv[4:]]
        date += [datetime.datetime.strptime(d, "%d.%m.%Y %H:%M").timetuple().tm_yday / 365 for d in date_str]
        size = len(date_str)
        city_one_hot += [city_encoder.transform([[city]]).toarray()[0]] * size
        cord += [city_attributes_raw.loc[city_attributes_raw["City"] == city][["Latitude", "Longitude"]].to_numpy()] * size

    set = {
        "d1": d1,
        "d2": d2,
        "d3": d3,
        "output_temp": output_temp,
        "output_wind": output_wind,
        "date": date,
        "city_one_hot": city_one_hot,
        "cord": cord
    }

    return {key: np.vstack(val).T for key, val in set.items()}

city_attributes_raw = pd.read_csv("data/city_attributes.csv", sep=";")

train_raw = load_train()
nearest_cities = get_nearest_cities(city_attributes_raw)
normalization_params = get_normalization_params(train_raw)
train_ctv = to_city_time_vect(train_raw)
# normalize(train_ctv, normalization_params)
# normalize_city_attributes(city_attributes_raw)
# wind_treshold = get_wind_treshold(6, normalization_params)
wind_treshold = 6
train_cdv = to_city_day_vect(train_ctv, wind_treshold)
city_encoder = get_city_encoder(city_attributes_raw)
train_set = get_set1(train_cdv, city_encoder, city_attributes_raw)
train_set = drop_nan_records(train_set)

test_raw = load_test()
test_ctv = to_city_time_vect(test_raw)
# normalize(test_ctv, normalization_params)
test_cdv = to_city_day_vect(test_ctv, wind_treshold)
test_set = get_set1(test_cdv, city_encoder, city_attributes_raw)
test_set = drop_nan_records(test_set)

In [2]:
import neural_network as nn
import definitions as d

rng = np.random.default_rng(1)

def get_day_layer(num):
    l = nn.InputLayer(120, f"d{num}")
    return nn.FullConnectLayer(l, 60, d.relu, rng)

def get_days_layer():
    ls = [get_day_layer(1), get_day_layer(2), get_day_layer(3)]
    l = nn.MergeLayer(ls)
    return nn.FullConnectLayer(l, 60, d.relu, rng)

def get_city_layer():
    coh = nn.InputLayer(36, "city_one_hot")
    date = nn.InputLayer(1, "date")
    cord = nn.InputLayer(2, "cord")
    return nn.MergeLayer([coh, date, cord])

def get_nn1(layer_sizes, activations, loss):
    assert len(layer_sizes) == len(activations)

    ds = get_days_layer()
    c = get_city_layer()
    l = nn.MergeLayer([ds, c])
    for (n, activation) in zip(layer_sizes, activations):
        l = nn.FullConnectLayer(l, n, activation, rng)
    return nn.NeuralNetwork(l, loss)

In [57]:
# net = get_nn1([60, 1], [d.relu, d.linear], d.l2_loss)
net.train(train_set, test_set, 1024, "output_temp", rng, 20)

Epoch 1/20: train: 31.62385197723188, test: 23485.307328350653
Epoch 2/20: train: 29.260145815524726, test: 22968.990651420903
Epoch 3/20: train: 27.1651623521407, test: 22339.535305635425
Epoch 4/20: train: 26.449341842627373, test: 21530.771097953664
Epoch 5/20: train: 24.877987234367993, test: 20405.465636839526
Epoch 6/20: train: 29.20296240047368, test: 19827.367007643672
Epoch 7/20: train: 33.525290962831605, test: 19278.692918999786
Epoch 8/20: train: 36.58663769006845, test: 19235.179602484375
Epoch 9/20: train: 34.9239913500306, test: 19839.166240569146
Epoch 10/20: train: 28.658604745497037, test: 20432.111005334416
Epoch 11/20: train: 36.67612266776217, test: 20659.092374308962
Epoch 12/20: train: 30.923622115089483, test: 21629.354503028753
Epoch 13/20: train: 30.463701492078346, test: 22472.5428982891
Epoch 14/20: train: 31.02414121921332, test: 23147.360456611328
Epoch 15/20: train: 33.68983248021466, test: 24200.10973756478
Epoch 16/20: train: 25.162787326400096, test: 2

In [61]:
predicted = net.propagate_forward(train_set)
print(predicted)
print(train_set["output_temp"])
diffs = np.abs(predicted - train_set["output_temp"])
print(np.min(diffs), np.max(diffs))
losses = d.l2_loss_n(predicted, train_set["output_temp"])
print(np.nanmin(losses), np.nanmax(losses))

[[286.65846424 286.88144833 284.45409479 ... 292.25359568 290.80737977
  290.01535794]]
[[287.7225     287.645      287.37541667 ... 293.57166667 291.07866108
  287.97708333]]
0.00038540078105597786 32.72176587866312
1.4853376203855777e-07 1070.713962218042


In [62]:
predicted = net.propagate_forward(test_set)
print(predicted)
print(test_set["output_temp"])
diffs = np.abs(predicted - test_set["output_temp"])
print(np.min(diffs), np.max(diffs))
losses = d.l2_loss_n(predicted, test_set["output_temp"])
print(np.nanmin(losses), np.nanmax(losses))

[[117.03424806 124.66707031 132.21020137 ... 126.113584   128.05420615
  125.42073462]]
[[274.52708333 271.91791667 269.57708333 ... 296.45708333 295.81458333
  294.15      ]]
111.30106701928415 220.15223261733348
12387.927519631181 48467.00552639651


In [ ]:
net2 = get_nn1([60, 20, 2], [d.relu, d.sigmoid, d.softmax], d.cross_entropy_loss)
net2.train(train_set, test_set, 1024, "output_wind", rng)

In [14]:
rng = np.random.default_rng(1)

def get_nn(layer_sizes, activations, loss):
    assert len(layer_sizes) == len(activations)

    d1_layer = nn.InputLayer(120, "d1")
    d2_layer = nn.InputLayer(120, "d2")
    d3_layer = nn.InputLayer(120, "d3")
    days_layer = nn.MergeLayer([d1_layer, d2_layer, d3_layer])

    coh_layer = nn.InputLayer(36, "city_one_hot")
    date_layer = nn.InputLayer(1, "date")
    cord_layer = nn.InputLayer(2, "cord")
    city_layer = nn.MergeLayer([coh_layer, date_layer, cord_layer])

    output_layer = nn.MergeLayer([days_layer, city_layer])
    for (n, activation) in zip(layer_sizes, activations):
        output_layer = nn.FullConnectLayer(output_layer, n, activation, rng)
    return nn.NeuralNetwork(output_layer, loss)

In [ ]:
# best ones:
net4 = get_nn([300, 120, 60, 1], [d.relu, d.sigmoid, d.relu, d.relu], d.l2_loss)            # mixed sigmoid and relu, no linear, L2 loss - 97,89% success rate
net4.train(train_set, test_set, 1024, "output_temp", rng, 50)

net3 = get_nn([300, 100, 60, 1, 1], [d.relu, d.relu, d.relu, d.relu, d.linear], d.l2_loss)  # only relu, linear at the end, L2 loss - 97,39% success rate
net3.train(train_set, test_set, 1024, "output_temp", rng, 50)

In [11]:
net3 = get_nn([300, 100, 60, 1, 1], [d.relu, d.relu, d.relu, d.relu, d.relu], d.l2_loss)
net3.train(train_set, test_set, 1024, "output_temp", rng, 50)

Epoch 1/50: train: 1.5878878735642612, test: 1.2696588783292804
Epoch 2/50: train: 1.073142299259767, test: 0.9473935972291473
Epoch 3/50: train: 0.8872722097740278, test: 0.8492196216065322
Epoch 4/50: train: 0.8403739133888967, test: 0.8277357501197931
Epoch 5/50: train: 0.8324270134271632, test: 0.8240305903352165
Epoch 6/50: train: 0.8315304871326011, test: 0.8234106671070144
Epoch 7/50: train: 0.8314578697860792, test: 0.8233027890730198
Epoch 8/50: train: 0.8314526037595962, test: 0.8232884858338345
Epoch 9/50: train: 0.8314529315076024, test: 0.8232894837671444
Epoch 10/50: train: 0.8314539379388374, test: 0.8232921521093747
Epoch 11/50: train: 0.8314550088150423, test: 0.823294893179031
Epoch 12/50: train: 0.8314560743979078, test: 0.8232976474284389
Epoch 13/50: train: 0.831457135169456, test: 0.8233004331575566
Epoch 14/50: train: 0.8314581949820992, test: 0.8233032592306062
Epoch 15/50: train: 0.8314592567603903, test: 0.823306132494718
Epoch 16/50: train: 0.8314603229537547

In [12]:
predicted = net3.predict(test_set)
# print(predicted)
# print(test_set["output_temp"])
diffs = np.abs(predicted - test_set["output_temp"])
print(f"min: {np.min(diffs)}, max: {np.max(diffs)}, mean: {np.mean(diffs)}, median: {np.median(diffs)}")
print(f"Good predictions: {np.count_nonzero(diffs <= 2)}, bad predictions: {np.count_nonzero(diffs > 2)}, success rate: {np.count_nonzero(diffs <= 2) / diffs.size * 100 : .2f}%")

min: 4.0568124653472815e-05, max: 3.909551528784863, mean: 0.7308811093893285, median: 0.6395367614404732
Good predictions: 12078, bad predictions: 333, success rate:  97.32%


In [122]:
net4 = get_nn([300, 120, 60, 60, 1], [d.relu, d.relu, d.relu, d.relu, d.linear], d.l2_loss)
net4.train(train_set, test_set, 1024, "output_temp", rng, 50)

Epoch 1/50: train: 0.78906344605059, test: 1.4343300022512013
Epoch 2/50: train: 0.12255857600500575, test: 1.5126048290925336
Epoch 3/50: train: 0.11337302371947092, test: 1.634899399760955
Epoch 4/50: train: 0.10698878177431567, test: 1.7508230918626693
Epoch 5/50: train: 0.10115390794197508, test: 1.7716065690906664
Epoch 6/50: train: 0.0983728152001137, test: 1.825186244539546
Epoch 7/50: train: 0.09428762490506061, test: 1.8867489169359895
Epoch 8/50: train: 0.09172895606472399, test: 1.9894601105198915
Epoch 9/50: train: 0.09087384109804907, test: 1.9670954647260688
Epoch 10/50: train: 0.088397059143718, test: 2.0500542740075525
Epoch 11/50: train: 0.08642064059158985, test: 2.126180877508857
Epoch 12/50: train: 0.08475278970750069, test: 2.2158284956124294
Epoch 13/50: train: 0.08339711285610989, test: 2.146735890450285
Epoch 14/50: train: 0.08388790243780016, test: 2.037982049629908
Epoch 15/50: train: 0.0833981688153626, test: 2.1152646587024537
Epoch 16/50: train: 0.080328179

In [123]:
predicted = net4.predict(test_set)
# print(predicted)
# print(test_set["output_temp"])
diffs = np.abs(predicted - test_set["output_temp"])
print(f"min: {np.min(diffs)}, max: {np.max(diffs)}, mean: {np.mean(diffs)}, median: {np.median(diffs)}")
print(f"Good predictions: {np.count_nonzero(diffs <= 2)}, bad predictions: {np.count_nonzero(diffs > 2)}, success rate: {np.count_nonzero(diffs <= 2) / diffs.size * 100 : .2f}%")

min: 0.00010808493156189147, max: 4.872049940411021, mean: 1.2025532296535677, median: 1.0479134727794892
Good predictions: 10239, bad predictions: 2172, success rate:  82.50%


In [105]:
net5 = get_nn([300, 140, 100, 40, 1], [d.relu, d.relu, d.relu, d.linear], d.l2_loss)
net5.train(train_set, test_set, 1024, "output_temp", rng, 50)

AssertionError: 

In [ ]:
predicted = net5.predict(test_set)
diffs = np.abs(predicted - test_set["output_temp"])
print(f"min: {np.min(diffs)}, max: {np.max(diffs)}, mean: {np.mean(diffs)}, median: {np.median(diffs)}")
print(f"Good predictions: {np.count_nonzero(diffs <= 2)}, bad predictions: {np.count_nonzero(diffs > 2)}, success rate: {np.count_nonzero(diffs <= 2) / diffs.size * 100 : .2f}%")

In [70]:
net3_wind = get_nn([300, 100, 60, 2, 2, 2], [d.sigmoid, d.sigmoid, d.relu, d.relu, d.sigmoid, d.softmax], d.cross_entropy_loss)
net3_wind.train(train_set, test_set, 1024, "output_wind", rng, 10)

Epoch 1/10: train: 0.5566873834380162, test: 0.720929590507864
Epoch 2/10: train: 0.5566442246647492, test: 0.7213692962454761
Epoch 3/10: train: 0.5566431902821923, test: 0.7213631926210489
Epoch 4/10: train: 0.5566433132408876, test: 0.7213190798676528
Epoch 5/10: train: 0.5566434326888592, test: 0.7212776756469556
Epoch 6/10: train: 0.5566435274256281, test: 0.7212416731512793
Epoch 7/10: train: 0.5566436067554377, test: 0.7212103608394359
Epoch 8/10: train: 0.5566436749625224, test: 0.721182940728686
Epoch 9/10: train: 0.5566437344419763, test: 0.7211587669686064
Epoch 10/10: train: 0.5566437868531428, test: 0.7211373248017345


In [71]:
predicted = net3_wind.predict(test_set)
print(predicted)
print(np.max(predicted, axis=1))
predicted = np.argmax(predicted, axis=0)
expected = test_set["output_wind"][0, :]
print(predicted)
print(expected)
print(np.count_nonzero(predicted == 1))
print(predicted.size)
print(f"Good predictions: {np.count_nonzero(predicted == expected)}, bad predictions: {np.count_nonzero(predicted != expected)}, success_rate: {np.count_nonzero(predicted == expected) / predicted.size * 100 : .2f}%")

[[0.24681464 0.24681445 0.24681449 ... 0.24681599 0.24681769 0.24681711]
 [0.75318536 0.75318555 0.75318551 ... 0.75318401 0.75318231 0.75318289]]
[0.2468225  0.75319455]
[1 1 1 ... 1 1 1]
[0 1 0 ... 1 0 0]
12411
12411
Good predictions: 4869, bad predictions: 7542, success_rate:  39.23%


In [4]:
def get_nn2(layer_sizes, activations, loss):
    assert len(layer_sizes) == len(activations)

    d1_layer = nn.InputLayer(120, "d1")
    d2_layer = nn.InputLayer(120, "d2")
    d3_layer = nn.InputLayer(120, "d3")
    coh_layer = nn.InputLayer(36, "city_one_hot")
    date_layer = nn.InputLayer(1, "date")
    cord_layer = nn.InputLayer(2, "cord")

    output_layer = nn.MergeLayer([d1_layer, d2_layer, d3_layer, coh_layer, date_layer, cord_layer])
    for (n, activation) in zip(layer_sizes, activations):
        output_layer = nn.FullConnectLayer(output_layer, n, activation, rng)
    return nn.NeuralNetwork(output_layer, loss)

In [20]:
# wn = get_nn2([2, 2], [d.sigmoid, d.softmax], d.hinge_loss)
wn.train(train_set, test_set, 1024, "output_wind", rng, 1)

Epoch 1/1: train: 1.1834330314038481, test: 1.1077163803938357


In [21]:
predicted = wn.predict(test_set)
print(predicted)
print(np.max(predicted, axis=1))
predicted = np.argmax(predicted, axis=0)
expected = test_set["output_wind"][0, :]
print(predicted)
print(expected)
print(np.count_nonzero(predicted == 1))
print(predicted.size)
print(f"Good predictions: {np.count_nonzero(predicted == expected)}, bad predictions: {np.count_nonzero(predicted != expected)}, success_rate: {np.count_nonzero(predicted == expected) / predicted.size * 100 : .2f}%")

[[0.39857106 0.3983715  0.40396978 ... 0.40522713 0.39855062 0.39825823]
 [0.60142894 0.6016285  0.59603022 ... 0.59477287 0.60144938 0.60174177]]
[0.88309602 0.6017803 ]
[1 1 1 ... 1 1 1]
[0 1 0 ... 1 0 0]
3744
12411
Good predictions: 7000, bad predictions: 5411, success_rate:  56.40%


In [98]:
print(d.hinge_loss_n(np.asarray([[0.6, 0.4], [0.4, 0.6]]), np.asarray([[0.0, 1.0], [1.0, 0.0]])))

[[1.2 0.8]
 [1.2 1.2]]
